In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error, r2_score
import warnings
import time
import os
warnings.filterwarnings('ignore')

In [9]:
directory = "../../datsets/Small LCL Data/"
finaldirectory = "../../datsets/LCL_Data_Transformed/"

def prepareDF(df, house):
    
    newdf = df.loc[df['LCLid'] == house]
    newdf = newdf.drop(['stdorToU'], axis=1)

    newdf['DateTime']= pd.to_datetime(newdf['DateTime'], format='%Y-%m-%d %H:%M')
    newdf = newdf.set_index(newdf['DateTime'])
    
    newdf = newdf[~newdf.index.duplicated(keep='first')]

    idx = pd.date_range(newdf.index.values[0], newdf.index.values[-1], freq='30min')
    
    meanUse = newdf["use"].mean()
    
    if len(idx) - len(newdf) > 10 or meanUse < 0.1 :
        #print(len(idx) - len(newdf))
        return newdf, False

    newdf = newdf.reindex(idx, fill_value=meanUse)

    newdf = newdf.resample('H').mean()
    newdf['month'] = newdf.index.month
    newdf['weekday'] = newdf.index.weekday
    newdf['hour'] = newdf.index.hour  

    return newdf, True



l = sorted([(filename, int(filename.split("_")[1][:-4])) for filename in os.listdir(directory)], key=lambda x: x[1])

temp=None
tempHouse = None
for filename, _ in l:
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        df = pd.read_csv(f, delimiter = ',')
        df = df.rename(columns={"KWH/hh (per half hour) ":"use"})
        df.drop(df.index[df['use'] == "Null"], inplace=True)
        #print(df.loc[df['use'] == "Null"])
        df['use'] = df['use'].astype(float)
        
        
        for house in df["LCLid"].unique():
            newdf, good = prepareDF(df, house)
            
            if not good:
                continue
            
            #last house, might want to join with 1st from following file
            if house == df["LCLid"].unique()[-1]:
                temp = newdf
                tempHouse = house
                continue
            
            #First house might want to join with last from prev file                
            #1sthouse == lasthouse 
            if house == tempHouse:
                #aggregate
                newdf = pd.concat([temp,newdf], ignore_index=False)
            
            newdf.to_csv(os.path.join(finaldirectory, house + ".csv"), index=True)
            
temp.to_csv(os.path.join(finaldirectory, tempHouse + ".csv"), index=True)#last file


../../datsets/Small LCL Data/LCL-June2015v2_0.csv
../../datsets/Small LCL Data/LCL-June2015v2_1.csv
../../datsets/Small LCL Data/LCL-June2015v2_2.csv
../../datsets/Small LCL Data/LCL-June2015v2_3.csv
../../datsets/Small LCL Data/LCL-June2015v2_4.csv
../../datsets/Small LCL Data/LCL-June2015v2_5.csv
../../datsets/Small LCL Data/LCL-June2015v2_6.csv
../../datsets/Small LCL Data/LCL-June2015v2_7.csv
../../datsets/Small LCL Data/LCL-June2015v2_8.csv
../../datsets/Small LCL Data/LCL-June2015v2_9.csv
../../datsets/Small LCL Data/LCL-June2015v2_10.csv
../../datsets/Small LCL Data/LCL-June2015v2_11.csv
../../datsets/Small LCL Data/LCL-June2015v2_12.csv
../../datsets/Small LCL Data/LCL-June2015v2_13.csv
../../datsets/Small LCL Data/LCL-June2015v2_14.csv
../../datsets/Small LCL Data/LCL-June2015v2_15.csv
../../datsets/Small LCL Data/LCL-June2015v2_16.csv
../../datsets/Small LCL Data/LCL-June2015v2_17.csv
../../datsets/Small LCL Data/LCL-June2015v2_18.csv
../../datsets/Small LCL Data/LCL-June2015

KeyboardInterrupt: 

In [ ]:
df1 = pd.read_csv("../../datsets/Small LCL Data/LCL-June2015v2_0.csv", delimiter = ',')


In [3]:
df1

,LCLid,stdorToU,DateTime,KWH/hh (per half hour)
0,MAC000002,Std,2012-10-12 00:30:00.0000000,0
1,MAC000002,Std,2012-10-12 01:00:00.0000000,0
2,MAC000002,Std,2012-10-12 01:30:00.0000000,0
3,MAC000002,Std,2012-10-12 02:00:00.0000000,0
4,MAC000002,Std,2012-10-12 02:30:00.0000000,0
...,...,...,...,...
999995,MAC000036,Std,2012-11-08 08:00:00.0000000,0.228
999996,MAC000036,Std,2012-11-08 08:30:00.0000000,0.042
999997,MAC000036,Std,2012-11-08 09:00:00.0000000,0.076
999998,MAC000036,Std,2012-11-08 09:30:00.0000000,0.07
